<a href="https://colab.research.google.com/github/rajasreekalli/Data-Visualization/blob/main/KNN_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [959]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from seaborn import heatmap
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, \
f1_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV

In [960]:
df = pd.read_csv('/content/drive/MyDrive/abalone.data')
df

,M,0.455,0.365,0.095,0.514,0.2245,0.101,0.15,15
0,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
1,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
2,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
3,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
4,I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.1200,8
...,...,...,...,...,...,...,...,...,...
4171,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4172,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4173,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4174,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [961]:
df = {'Length': [0.455, 0.350, 0.530, 0.440, 0.330, 0.425, 0.565, 0.590, 0.600, 0.625, 0.710],
        'Diameter' : [0.365, 0.265, 0.420, 0.365, 0.255, 0.300, 0.450, 0.440, 0.475, 0.485, 0.555],
        'Height' : [0.095, 0.090, 0.135, 0.125, 0.080, 0.095, 0.165, 0.135, 0.205, 0.150, 0.195],
        'Whole weight' : [0.512, 0.2255, 0.6770, 0.5160, 0.2050, 0.3515, 0.8870, 0.9660, 1.1760, 1.0945, 1.9485],
        'Shucked weight' : [0.2245, 0.0995, 0.2565, 0.2155, 0.0895, 0.1410, 0.3700, 0.4390, 0.5255, 0.5310, 0.9455],
        'Viscera weight' : [0.101, 0.0485, 0.1415, 0.1140, 0.0395, 0.0775, 0.2390, 0.2145, 0.2875, 0.2610, 0.3765],
        'Shell weight' : [0.15, 0.070, 0.210, 0.155, 0.055, 0.120, 0.2490, 0.2605, 0.3080, 0.2960, 0.4950],
        'Sex' : ['m', 'm', 'f', 'm', 'i', 'i', 'f', 'm', 'm', 'f', 'm'],
        'Age' : [16.5, 8.5, 10.5, 11.5, 8.5, 9.5, 12.5, 11.5, 10.5, 11.5, 13.5],
        'Rings' : [15, 7, 9, 10, 7, 8, 11, 10, 9, 10, 12]}
df = pd.DataFrame(df)
df


,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Sex,Age,Rings
0,0.455,0.365,0.095,0.5120,0.2245,0.1010,0.1500,m,16.5,15
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,m,8.5,7
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,f,10.5,9
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,m,11.5,10
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,i,8.5,7
5,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.1200,i,9.5,8
6,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,f,12.5,11
7,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,m,11.5,10
8,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,m,10.5,9
9,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,f,11.5,10


In [962]:
df.dtypes

Length            float64
Diameter          float64
Height            float64
Whole weight      float64
Shucked weight    float64
Viscera weight    float64
Shell weight      float64
Sex                object
Age               float64
Rings               int64
dtype: object

In [963]:
df.duplicated().any()

False

In [964]:
print('Missing data:\n', df.isna().sum())
print('\n')
print('Duplicates:', df.duplicated().sum())

Missing data:
 Length            0
Diameter          0
Height            0
Whole weight      0
Shucked weight    0
Viscera weight    0
Shell weight      0
Sex               0
Age               0
Rings             0
dtype: int64


Duplicates: 0


In [965]:
# 1) Is this a balanced data set?
#  split data into train and test sets...notice that stratify = y will give the same proportion of yes/no in our train and test sets
X = df.drop(columns = ['Age', 'Sex'])
y = df['Sex']

In [966]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)


In [967]:
print(X.shape)
print(y.shape)

(11, 8)
(11,)


In [968]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [969]:
knn = KNeighborsClassifier()

knn.fit(X_train_scaled, y_train)

train_preds = knn.predict(X_train_scaled)
test_preds = knn.predict(X_test_scaled)

In [970]:
print(knn)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')


In [971]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [972]:
predictions = knn.predict(X_test)
predictions

array(['f', 'f', 'f'], dtype=object)

In [973]:
pipe = make_pipeline(scaler, knn)
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('kneighborsclassifier',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=5, p=2,
                                      weights='uniform'))],
         verbose=False)

No this is not a balanced data set. So we have to explore it.

In [974]:
# 2) Be sure to scale your data (try using the pipeline!)
pipe = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors = 5))

In [975]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('kneighborsclassifier',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=5, p=2,
                                      weights='uniform'))],
         verbose=False)

In [976]:
pipe.score(X_train, y_train)
pipe.score(X_test, y_test)

0.6666666666666666

In [977]:
# 3) What is the accuracy of your default model?
def evaluate_classification(y_true, y_test, average=None):
  accuracy = accuracy_score(y_true, y_test)
  recall = recall_score(y_true, y_test, average=average)
  precision = precision_score(y_true, y_test, average=average)
  f1 = f1_score(y_true, y_test, average=average)

  print(f'Accuracy: {accuracy} \nRecall: {recall} \nPrecision: {precision} \nF1 Score: {f1}')

In [978]:
print('Training Metrics\n')
evaluate_classification(y_train, train_preds, average='macro')
print('\n')

print('Testing Metrics\n')
evaluate_classification(y_test, test_preds, average='macro')

Training Metrics

Accuracy: 0.625 
Recall: 0.75 
Precision: 0.7222222222222222 
F1 Score: 0.6222222222222222


Testing Metrics

Accuracy: 0.6666666666666666 
Recall: 0.5 
Precision: 0.3333333333333333 
F1 Score: 0.4


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [979]:
# 4) What is the optimal accuracy you can achieve by tuning the value of K using KNN?
print('Training Accuracy', pipe.score(X_train, y_train))

Training Accuracy 0.625


In [980]:
print('Testing Accuracy', pipe.score(X_test, y_test))

Testing Accuracy 0.6666666666666666


In [981]:
# 5) Create a confusion matrix for you optimal KNN model.
print('Classification Report for Training Set')
train_report = classification_report(y_train, train_preds)
print(train_report)

print('Confusion Matrix for Training Set')
train_conf_mat = confusion_matrix(y_train, train_preds)
print(train_conf_mat)

print('\n\n')

print('Classification Report for Testing Set')

test_report = classification_report(y_test, test_preds)
print(test_report)

print('Confusion Matrix for Testing Set')
test_conf_mat = confusion_matrix(y_test, test_preds)
print(test_conf_mat)

Classification Report for Training Set
              precision    recall  f1-score   support

           f       0.50      1.00      0.67         2
           i       0.67      1.00      0.80         2
           m       1.00      0.25      0.40         4

    accuracy                           0.62         8
   macro avg       0.72      0.75      0.62         8
weighted avg       0.79      0.62      0.57         8

Confusion Matrix for Training Set
[[2 0 0]
 [0 2 0]
 [2 1 1]]



Classification Report for Testing Set
              precision    recall  f1-score   support

           f       0.00      0.00      0.00         1
           m       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3

Confusion Matrix for Testing Set
[[0 1]
 [0 2]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
